## Clean CSV

In [ ]:
import pandas as pd

o_prefix = "city"

stops_df = pd.read_csv("./data/stops.csv")
stops_df = stops_df[["stop_id", "stop_name", "stop_lat", "stop_lon"]]
stops_df = stops_df.rename(columns=lambda x: x.replace("stop_", ""))
stops_df = stops_df.rename(columns={"name": f"{o_prefix}.address",
                                    "lat": f"{o_prefix}.lat",
                                    "lon": f"{o_prefix}.lon"})

## Add Coords from Google

In [ ]:
from time import sleep
from utils import get_bus_station, get_place_latlon, get_dist

from env import GOOGLE_API_KEY

In [ ]:
g_prefix = "google"
stops_df[[f"{g_prefix}.lat",
          f"{g_prefix}.lon",
          f"{o_prefix}_{g_prefix}.distance"]] = [0.0,0.0,0.0]

In [ ]:
for idx,row in stops_df.iterrows():
  if idx%100 == 0:
    print(idx)
    stops_df.to_csv("./data/stops.tgh.temp.csv", index=False)

  if abs(row[f"{g_prefix}.lat"]) < 1.0 and abs(row[f"{g_prefix}.lon"]) < 1.0:
    response = get_bus_station(GOOGLE_API_KEY, lat=row[f"{o_prefix}.lat"], lon=row[f"{o_prefix}.lon"], radius=50)
    if "places" in response:
      station = response["places"][0]
      station_latlon = get_place_latlon(station)
      distance = get_dist((row[f"{o_prefix}.lat"], row[f"{o_prefix}.lon"]), station_latlon)
      stops_df.loc[idx, f"{g_prefix}.lat"] = station_latlon[0]
      stops_df.loc[idx, f"{g_prefix}.lon"] = station_latlon[1]
      stops_df.loc[idx, f"{o_prefix}_{g_prefix}.distance"] = distance
    sleep(0.1)

In [ ]:
stops_df.to_csv("./data/stops.tgh.csv", index=False)

## Street View

### Import and Helpers

In [ ]:
import pandas as pd

from os import makedirs, path
from time import sleep

from utils import get_address, get_latlon
from utils import get_heading, get_streetview_image

from env import GOOGLE_API_KEY

for dir in ["address", "address_coords", "heading", "station_coords"]:
  makedirs(f"./imgs/{dir}", exist_ok=True)

### Test coords vs address

In [ ]:
stops_df = pd.read_csv("./data/stops.tgh.csv")
stops_df

In [ ]:
o_prefix = "city"
g_prefix = "google"

for idx,row in stops_df.iterrows():
  if idx > 2047:
    break

  fname = str(row["id"]) + ".jpg"
  address_path = path.join("./imgs/address", fname)
  address_coords_path = path.join("./imgs/address_coords", fname)
  heading_path = path.join("./imgs/heading", fname)
  station_coords_path = path.join("./imgs/station_coords", fname)

  csv_lat, csv_lon = row[f"{o_prefix}.lat"], row[f"{o_prefix}.lon"]

  if not path.isfile(address_path):
    address_img = get_streetview_image(GOOGLE_API_KEY, address=row[f"{o_prefix}.address"])
    address_img.save(address_path)
    sleep(0.1)

  if not path.isfile(address_coords_path):
    coords_img = get_streetview_image(GOOGLE_API_KEY, lat=csv_lat, lon=csv_lon)
    coords_img.save(address_coords_path)
    sleep(0.1)

  if path.isfile(heading_path) and path.isfile(station_coords_path):
    continue

  station_lat, station_lon = row[f"{g_prefix}.lat"], row[f"{g_prefix}.lon"]
  csv_g_dist = row[f"{o_prefix}_{g_prefix}.distance"]

  if abs(station_lat) < 1.0 and abs(station_lon) < 1.0:
    continue

  if csv_g_dist > 50.0:
    continue

  if not path.isfile(heading_path):
    heading = get_heading((csv_lat, csv_lon), (station_lat, station_lon))
    coords_img = get_streetview_image(GOOGLE_API_KEY, lat=csv_lat, lon=csv_lon, heading=heading)
    coords_img.save(heading_path)
    sleep(0.1)

  if not path.isfile(station_coords_path):
    scoord_img = get_streetview_image(GOOGLE_API_KEY, lat=station_lat, lon=station_lon)
    scoord_img.save(station_coords_path)
    sleep(0.1)

In [ ]:
row = stops_df.iloc[0]
display(row)
display(get_address(GOOGLE_API_KEY, lat=row["lat"], lon=row["lon"]))
display(get_latlon(GOOGLE_API_KEY, address=f"{row['address']}, Fortaleza, Ceara, Brazil"))
display(get_bus_station(GOOGLE_API_KEY, lat=row["lat"], lon=row["lon"], radius=8))

In [ ]:
places = get_bus_station(GOOGLE_API_KEY, lat=row["lat"], lon=row["lon"], radius=8)["places"]
bus_station_latlon = get_place_latlon(places[0])
get_heading((row["lat"], row["lon"]), bus_station_latlon)